# Lab | Random Forests

- Apply the Random Forest algorithm to predict the TARGET_B. Please note that this column suffers from class imbalance. Fix the class imbalance using upsampling.
- Discuss the model predictions and it's impact in the business scenario. Is the cost of a false positive equal to the cost of the false negative? How much the money the company will not earn because of missclassifications made by the model?
- Sklearn classification models are trained to maximize the accuracy. However, another error metric will be more relevant here. Which one? Please checkout make_scorer alongside with GridSearchCV in order to train the model to maximize the error metric of interest in this case.

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np

## Reading in the data

In [2]:
cat = pd.read_csv('files_for_lab/categorical.csv')

In [3]:
num = pd.read_csv('files_for_lab/numerical.csv')

In [4]:
target = pd.read_csv('files_for_lab/target.csv')

In [ ]:
cat.head()

In [ ]:
num.head()

In [ ]:
target.head()

### Checking cardinality in categorical columns

In [5]:
for col in cat:
    print(col, cat[col].nunique())

STATE 12
CLUSTER 53
HOMEOWNR 2
GENDER 3
DATASRCE 3
RFA_2R 1
RFA_2A 4
GEOCODE2 4
DOMAIN_A 5
DOMAIN_B 4
ODATEW_YR 15
ODATEW_MM 12
DOB_YR 96
DOB_MM 12
MINRDATE_YR 20
MINRDATE_MM 12
MAXRDATE_YR 18
MAXRDATE_MM 12
LASTDATE_YR 3
LASTDATE_MM 12
FIRSTDATE_YR 26
FIRSTDATE_MM 12


We will group the states into groups to reduce cardinality and remove other columns with high cardinality.

In [6]:
cat['STATE'].value_counts()

STATE
other    30457
CA       17343
FL        8376
TX        7535
IL        6420
MI        5654
NC        4160
WA        3577
GA        3403
IN        2980
WI        2795
MO        2712
Name: count, dtype: int64

In [7]:
regions = {'east':['FL', 'NC', 'GA'],
           'central':['TX', 'IL', 'MI', 'IN', 'WI', 'MO'],
           'west':['CA', 'WA'],
           'other':['other']
          }

In [8]:
def get_key(val):
    for key, value in regions.items():
        for state in value:
            if val == state:
                return key


In [9]:
cat['regions'] = cat['STATE'].apply(get_key)

In [10]:
cat.head()

,STATE,CLUSTER,HOMEOWNR,GENDER,DATASRCE,RFA_2R,RFA_2A,GEOCODE2,DOMAIN_A,DOMAIN_B,ODATEW_YR,ODATEW_MM,DOB_YR,DOB_MM,MINRDATE_YR,MINRDATE_MM,MAXRDATE_YR,MAXRDATE_MM,LASTDATE_YR,LASTDATE_MM,FIRSTDATE_YR,FIRSTDATE_MM,regions
0,IL,36,H,F,3,L,E,C,T,2,89,1,37,12,92,8,94,2,95,12,89,11,central
1,CA,14,H,M,3,L,G,A,S,1,94,1,52,2,93,10,95,12,95,12,93,10,west
2,NC,43,U,M,3,L,E,C,R,2,90,1,0,2,91,11,92,7,95,12,90,1,east
3,CA,44,U,F,3,L,E,C,R,2,87,1,28,1,87,11,94,11,95,12,87,2,west
4,FL,16,H,F,3,L,F,A,S,2,86,1,20,1,93,10,96,1,96,1,79,3,east


In [11]:
cat['regions'].nunique()

4

In [12]:
cat = cat.drop('STATE', axis=1)

In [13]:
cat.columns

Index(['CLUSTER', 'HOMEOWNR', 'GENDER', 'DATASRCE', 'RFA_2R', 'RFA_2A',
       'GEOCODE2', 'DOMAIN_A', 'DOMAIN_B', 'ODATEW_YR', 'ODATEW_MM', 'DOB_YR',
       'DOB_MM', 'MINRDATE_YR', 'MINRDATE_MM', 'MAXRDATE_YR', 'MAXRDATE_MM',
       'LASTDATE_YR', 'LASTDATE_MM', 'FIRSTDATE_YR', 'FIRSTDATE_MM',
       'regions'],
      dtype='object')

### Removing columns with dates

In [14]:
cols_to_drop = ['CLUSTER', 'ODATEW_YR', 'ODATEW_MM', 'DOB_YR',
       'DOB_MM', 'MINRDATE_YR', 'MINRDATE_MM', 'MAXRDATE_YR', 'MAXRDATE_MM',
       'LASTDATE_YR', 'LASTDATE_MM', 'FIRSTDATE_YR', 'FIRSTDATE_MM']
cat.drop(cols_to_drop, axis = 1, inplace = True)


## Combining data

In [15]:
data = pd.concat([cat, num, target], axis=1)

In [16]:
data.head()

,HOMEOWNR,GENDER,DATASRCE,RFA_2R,RFA_2A,GEOCODE2,DOMAIN_A,DOMAIN_B,regions,TCODE,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,STATEGOV,FEDGOV,WEALTH2,POP901,POP902,POP903,POP90C1,POP90C2,POP90C3,POP90C4,POP90C5,ETH1,ETH2,ETH3,ETH4,ETH5,ETH6,ETH7,ETH8,ETH9,ETH10,ETH11,ETH12,ETH13,ETH14,ETH15,ETH16,AGE901,AGE902,AGE903,AGE904,AGE905,AGE906,AGE907,CHIL1,CHIL2,CHIL3,AGEC1,AGEC2,AGEC3,AGEC4,AGEC5,AGEC6,AGEC7,CHILC1,CHILC2,CHILC3,CHILC4,CHILC5,HHAGE1,HHAGE2,HHAGE3,HHN1,HHN2,HHN3,HHN4,HHN5,HHN6,MARR1,MARR2,MARR3,MARR4,HHP1,HHP2,DW1,DW2,DW3,DW4,DW5,DW6,DW7,DW8,DW9,HV1,HV2,HV3,HV4,HU1,HU2,HU3,HU4,HU5,HHD1,HHD2,HHD3,HHD4,HHD5,HHD6,HHD7,HHD8,HHD9,HHD10,HHD11,HHD12,ETHC1,ETHC2,ETHC3,ETHC4,ETHC5,ETHC6,HVP1,HVP2,HVP3,HVP4,HVP5,HVP6,HUR1,HUR2,RHP1,RHP2,RHP3,RHP4,HUPA1,HUPA2,HUPA3,HUPA4,HUPA5,HUPA6,HUPA7,RP1,RP2,RP3,RP4,MSA,ADI,DMA,IC1,IC2,IC3,IC4,IC5,IC6,IC7,IC8,IC9,IC10,IC11,IC12,IC13,IC14,IC15,IC16,IC17,IC18,IC19,IC20,IC21,IC22,IC23,HHAS1,HHAS2,HHAS3,HHAS4,MC1,MC2,MC3,TPE1,TPE2,TPE3,TPE4,TPE5,TPE6,TPE7,TPE8,TPE9,PEC1,PEC2,TPE10,TPE11,TPE12,TPE13,LFC1,LFC2,LFC3,LFC4,LFC5,LFC6,LFC7,LFC8,LFC9,LFC10,OCC1,OCC2,OCC3,OCC4,OCC5,OCC6,OCC7,OCC8,OCC9,OCC10,OCC11,OCC12,OCC13,EIC1,EIC2,EIC3,EIC4,EIC5,EIC6,EIC7,EIC8,EIC9,EIC10,EIC11,EIC12,EIC13,EIC14,EIC15,EIC16,OEDC1,OEDC2,OEDC3,OEDC4,OEDC5,OEDC6,OEDC7,EC1,EC2,EC3,EC4,EC5,EC6,EC7,EC8,SEC1,SEC2,SEC3,SEC4,SEC5,AFC1,AFC2,AFC3,AFC4,AFC5,AFC6,VC1,VC2,VC3,VC4,ANC1,ANC2,ANC3,ANC4,ANC5,ANC6,ANC7,ANC8,ANC9,ANC10,ANC11,ANC12,ANC13,ANC14,ANC15,POBC1,POBC2,LSC1,LSC2,LSC3,LSC4,VOC1,VOC2,VOC3,HC1,HC2,HC3,HC4,HC5,HC6,HC7,HC8,HC9,HC10,HC11,HC12,HC13,HC14,HC15,HC16,HC17,HC18,HC19,HC20,HC21,MHUC1,MHUC2,AC1,AC2,CARDPROM,NUMPROM,CARDPM12,NUMPRM12,RAMNTALL,NGIFTALL,CARDGIFT,MINRAMNT,MAXRAMNT,LASTGIFT,TIMELAG,AVGGIFT,CONTROLN,HPHONE_D,RFA_2F,CLUSTER2,TARGET_B,TARGET_D
0,H,F,3,L,E,C,T,2,central,0,60.000000,5,9,0,0,39,34,18,10,2,1,5,992,264,332,0,35,65,47,53,92,1,0,0,11,0,0,0,0,0,0,0,11,0,0,0,39,48,51,40,50,54,25,31,42,27,11,14,18,17,13,11,15,12,11,34,25,18,26,10,23,18,33,49,28,12,4,61,7,12,19,198,276,97,95,2,2,0,0,7,7,0,479,635,3,2,86,14,96,4,7,38,80,70,32,84,16,6,2,5,9,15,3,17,50,25,0,0,0,2,7,13,27,47,0,1,61,58,61,15,4,2,0,0,14,1,0,0,2,5,17,73,0.0,177.0,682.0,307,318,349,378,12883,13,23,23,23,15,1,0,0,1,4,25,24,26,17,2,0,0,2,28,4,51,1,46,54,3,88,8,0,0,0,0,0,0,4,1,13,14,16,2,45,56,64,50,64,44,62,53,99,0,0,9,3,8,13,9,0,3,9,3,15,19,5,4,3,0,3,41,1,0,7,13,6,5,0,4,9,4,1,3,10,2,1,7,78,2,0,120,16,10,39,21,8,4,3,5,20,3,19,4,0,0,0,18,39,0,34,23,18,16,1,4,0,23,0,0,5,1,0,0,0,0,0,2,0,3,74,88,8,0,4,96,77,19,13,31,5,14,14,31,54,46,0,0,90,0,10,0,0,0,33,65,40,99,99,6,2,10,7,27,74,6,14,240.0,31,14,5.0,12.0,10.0,4,7.741935,95515,0,4,39,0,0.0
1,H,M,3,L,G,A,S,1,west,1,46.000000,6,9,16,0,15,55,11,6,2,1,9,3611,940,998,99,0,0,50,50,67,0,0,31,6,4,2,6,4,14,0,0,2,0,1,4,34,41,43,32,42,45,32,33,46,21,13,14,33,23,10,4,2,11,16,36,22,15,12,1,5,4,21,75,55,23,9,69,4,3,24,317,360,99,99,0,0,0,0,0,0,0,5468,5218,12,10,96,4,97,3,9,59,94,88,55,95,5,4,1,3,5,4,2,18,44,5,0,0,0,97,98,98,98,99,94,0,83,76,73,21,5,0,0,0,4,0,0,0,91,91,91,94,4480.0,13.0,803.0,1088,1096,1026,1037,36175,2,6,2,5,15,14,13,10,33,2,5,2,5,15,14,14,10,32,6,2,66,3,56,44,9,80,14,0,0,0,0,0,0,6,0,2,24,32,12,71,70,83,58,81,57,64,57,99,99,0,22,24,4,21,13,2,1,6,0,4,1,0,3,1,0,6,13,1,2,8,18,11,4,3,4,10,7,11,1,6,2,1,16,69,5,2,160,5,5,12,21,7,30,20,14,24,4,24,10,0,0,0,8,15,0,55,10,11,0,0,2,0,3,1,1,2,3,1,1,0,3,0,0,0,42,39,50,7,27,16,99,92,53,5,10,2,26,56,97,99,0,0,0,96,0,4,0,0,0,99,0,99,99,99,20,4,6,5,12,32,6,13,47.0,3,1,10.0,25.0,25.0,18,15.666667,148535,0,2,1,0,0.0
2,U,M,3,L,E,C,R,2,east,1,61.611649,3,1,2,0,20,29,33,6,8,1,1,7001,2040,2669,0,2,98,49,51,96,2,0,0,2,0,0,0,0,0,0,0,2,0,0,0,35,43,46,37,45,49,23,35,40,25,13,20,19,16,13,10,8,15,14,30,22,19,25,10,23,21,35,44,22,6,2,63,9,9,19,183,254,69,69,1,6,5,3,3,3,0,497,546,2,1,78,22,93,7,18,36,76,65,30,86,14,7,2,5,11,17,3,17,60,18,0,1,0,0,1,6,18,50,0,4,36,49,51,14,5,4,2,24,11,2,3,6,0,2,9,44,0.0,281.0,518.0,251,292,292,340,11576,32,18,20,15,

In [17]:
### checking for nan
data.isna().sum().loc[lambda x: x > 0]

Series([], dtype: int64)

## Train test split

In [18]:
from sklearn.model_selection import train_test_split

X = data.drop(['TARGET_B'], axis =1)
y = data['TARGET_B']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


In [19]:
#concat X_train and y_train temporarily for upsampling
trainset = pd.concat([X_train, y_train], axis = 1)

In [20]:
trainset['TARGET_B'].value_counts()

TARGET_B
0    72486
1     3843
Name: count, dtype: int64

In [21]:
from sklearn.utils import resample

category_0 = trainset[trainset['TARGET_B']==0]

category_1 = trainset[trainset['TARGET_B']==1]

category_1_resample = resample(category_1, n_samples=len(category_0), replace = True, random_state=0)


train_resampled_df = pd.concat([category_0, category_1_resample], axis=0, ignore_index=True)
train_resampled_df = train_resampled_df.sample(frac=1)

X_train = train_resampled_df.drop(['TARGET_B'], axis =1)
y_train = train_resampled_df['TARGET_B']

In [22]:
X_train = pd.DataFrame(X_train)
X_test = pd.DataFrame(X_test)

y_train_regression = X_train['TARGET_D']
y_test_regression = X_test['TARGET_D']

# Now we can remove the column target d from the set of features
X_train = X_train.drop(['TARGET_D'], axis = 1)
X_test = X_test.drop(['TARGET_D'], axis = 1)

## Building the pipeline

In [23]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PowerTransformer, StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import cohen_kappa_score, make_scorer

numeric_features = X_train.select_dtypes(np.number).columns
categorical_features = X_train.select_dtypes('object').columns

# We define a pipeline to preprocess the data
numeric_transformer = Pipeline(steps=[
    ("transfomer", PowerTransformer()),
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('encoder', OneHotEncoder())
])


# We define a pipeline to perform the preprocessing
preprocessor = ColumnTransformer(transformers=[
    ("num_transform", numeric_transformer, numeric_features),
    ("cat_transform", categorical_transformer, categorical_features)
]) #num_features can be a list of specific columns

# We define the full pipeline
pipeline = Pipeline(steps=[("preprocesser", preprocessor), ("RandomForest", RandomForestClassifier())])
pipeline

Pipeline(steps=[('preprocesser',
                 ColumnTransformer(transformers=[('num_transform',
                                                  Pipeline(steps=[('transfomer',
                                                                   PowerTransformer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index(['DATASRCE', 'DOMAIN_B', 'TCODE', 'AGE', 'INCOME', 'WEALTH1', 'HIT',
       'MALEMILI', 'MALEVET', 'VIETVETS',
       ...
       'CARDGIFT', 'MINRAMNT', 'MAXRAMNT', 'LASTGIFT', 'TIMELAG', 'AVGGIFT',
       'CONTROLN', 'HPHONE_D', 'RFA_2F', 'CLUSTER2'],
      dtype='object', length=317)),
                                                 ('cat_transform',
                                                  Pipeline(steps=[('encoder',
                                                                   OneHotEncoder())]),
                                                  Index(['HOMEOWNR', 'GENDER', 'RFA_2R', 'RFA_2A', 'GEOCODE2', 'DOMAIN_A',
       'regions'],
      dtype='object'))])),
                ('RandomForest', RandomForestClassifier())])

In [57]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocesser',
                 ColumnTransformer(transformers=[('num_transform',
                                                  Pipeline(steps=[('transfomer',
                                                                   PowerTransformer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index(['DATASRCE', 'DOMAIN_B', 'TCODE', 'AGE', 'INCOME', 'WEALTH1', 'HIT',
       'MALEMILI', 'MALEVET', 'VIETVETS',
       ...
       'CARDGIFT', 'MINRAMNT', 'MAXRAMNT', 'LASTGIFT', 'TIMELAG', 'AVGGIFT',
       'CONTROLN', 'HPHONE_D', 'RFA_2F', 'CLUSTER2'],
      dtype='object', length=317)),
                                                 ('cat_transform',
                                                  Pipeline(steps=[('encoder',
                                                                   OneHotEncoder())]),
                                                  Index(['HOMEOWNR', 'GENDER', 'RFA_2R', 'RFA_2A', 'GEOCODE2', 'DOMAIN_A',
       'regions'],
      dtype='object'))])),
                ('RandomForest', RandomForestClassifier())])

In [74]:
#save the pipeline
import pickle

filename = "files_for_lab/rf_pipeline.pkl"

with open(filename, "wb") as file:
    pickle.dump(pipeline, file)

In [58]:
pipeline.score(X_test, y_test)

0.9457632447728345

In [61]:
print("The Accuracy for the Random Forest in the TRAIN set is {:.2f}".format(pipeline.score(X_train, y_train)))
print("The Accuracy for the Random Forest in the TEST  set is {:.2f}".format(pipeline.score(X_test, y_test)))

The Accuracy for the Random Forest in the TRAIN set is 1.00
The Accuracy for the Random Forest in the TEST  set is 0.95


In [59]:
y_test_pred = pipeline.predict(X_test)
y_test_pred

array([0, 0, 0, ..., 0, 0, 0])

In [66]:
y_test_pred_df = pd.DataFrame(y_test_pred, columns=['target_b_predictions'], index=X_test.index)
y_test_pred_df

,target_b_predictions
58053,0
9484,0
13395,0
1466,0
2076,0
...,...
94255,0
26449,0
1969,0
48574,0


In [63]:
from sklearn.metrics import confusion_matrix

cm_test = confusion_matrix(y_test, y_test_pred)
cm_test

array([[18046,    37],
       [  998,     2]])

In [67]:
# y=0, y_pred=0 -> earning = 0
# y=0, y_pred=1 -> earning = 0
# y=1, y_pred=1 -> earning = target_d
# y=1, y_pred=0 -> earning = -target_d

earnings = pd.concat([y_test,y_test_pred_df, y_test_regression], axis=1)

In [71]:
true_positive = earnings[(earnings['TARGET_B']==1) & (earnings['target_b_predictions']==1)]
true_positive

,TARGET_B,target_b_predictions,TARGET_D
3043,1,1,15.0
79003,1,1,5.0


In [72]:
false_negative = earnings[(earnings['TARGET_B']==1) & (earnings['target_b_predictions']==0)]
false_negative

,TARGET_B,target_b_predictions,TARGET_D
19972,1,0,5.0
19307,1,0,12.0
76808,1,0,50.0
36712,1,0,25.0
9634,1,0,10.0
...,...,...,...
85869,1,0,25.0
48596,1,0,3.0
10266,1,0,4.0
82843,1,0,11.0


In [73]:
false_positive['TARGET_D'].sum()

16081.03

## Consequences

The business impact of using this model is pretty dire: the model only accurately predicts two donations. It mislables 998 rows of donors as non-donors, which is essentially all of their donor group. This means they don't expect any donations, and they get 16k. Planning isn't possible this way.

So the score that we want to use to evaluate our model is the score that evaluates the number of false negatives - 100% recall means no false negatives.

Recall: the number of instances which the model correctly identified as relevant out of the total relevant instances.

## Using pipeline and gridsearch for best model and better scoring

I tried running this code with several choices of parameters but after two hours it hadn't terminated. I tried running it again with just one parameter but didn't have time to let it run.

In [26]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PowerTransformer, StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier

numeric_features = X_train.select_dtypes(np.number).columns
categorical_features = X_train.select_dtypes('object').columns

# We define a pipeline to preprocess the data
numeric_transformer = Pipeline(steps=[
    ("transfomer", PowerTransformer()),
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('encoder', OneHotEncoder())
])


# We define a pipeline to perform the preprocessing
preprocessor = ColumnTransformer(transformers=[
    ("num_transform", numeric_transformer, numeric_features),
    ("cat_transform", categorical_transformer, categorical_features)
]) #num_features can be a list of specific columns

# We define the full pipeline
pipeline = Pipeline(steps=[("preprocesser", preprocessor), ("RandomForest", RandomForestClassifier())])
pipeline

# import Grid Search class
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import recall_score, make_scorer

#make score
recall_scorer = make_scorer(recall_score)

# make lists of different parameters to check
param_grid = {
    'RandomForest__criterion':['gini'],
    'RandomForest__max_depth': [10],
    'RandomForest__max_features': ['sqrt'],
    'RandomForest__min_samples_leaf': [2],
    'RandomForest__min_samples_split': [5]
  }
# initialize
grid_pipeline = GridSearchCV(pipeline,param_grid, cv=5, return_train_score=True, scoring=recall_scorer)


# fit
grid_pipeline.fit(X_train,y_train)
grid_pipeline.best_params_
grid_pipleine.best_score_

KeyboardInterrupt: 